# Анализ мошенничества с кредитными картами
<br>**Описание задачи**
<br>Данные представляют собой транзакции, совершенные с помощью кредитных карт в сентябре
2013 года европейскими держателями карт. <br>В датасете содержится информация о 284,807
транзакциях, включая 492 случая мошенничества, что составляет 0.172% от общего числа
транзакций. <br>Данные представлены числовыми переменными, полученными в результате
преобразования методом главных компонент (PCA). <br>Оригинальные признаки и подробности о
данных не предоставлены из соображений конфиденциальности. <br>Признаки V1-V28 являются
результатом PCA, <br>'Time' содержит время в секундах между транзакцией и первой записью,
<br>а 'Amount' обозначает сумму транзакции.

**Ход решения**
1. Корреляционный анализ: 
    - Рассчитать корреляционную матрицу для числовых переменных. 
    - Возвращаемый тип данных - numpy массив с размерностью 2.
2. Проверка статистических гипотез:   
  - Проверить гипотезы, например, о равенстве средних сумм для признаков.   
  - Параметры включают признаки для сравнения, критерий проверки (t-test, z-test, kolmogorov-smirnov, fisher) и уровень значимости.
3. Точечные оценки параметров распределени  я: 
  - Оценить средние и дисперсии для выбранных признаков с использованием метода моментов и метода максимального правдоподоб  ия. 
  - Возвращаемый тип данных - строка с оценками.
4. Определение влияния признаков на безопасность транза  кции: 
  - Применить логистическую регрессию для выявления признаков, оказывающих наибольшее влияние на безопасность транз  акций. 
  - Возвращаемый тип данных - список с названиями признаков на основе весов в логистической регрессии, ограниченный top_k параметрами.
5. Интервальные   оценки: 
  - Рассчитать доверительные интервалы для среднего значения выбранного признака с учетом уровня зн  ачимости. 
  - Использовать t- или z-статистику в зависимости от размера выборки (подсказка: центральная предельная теорема и зна  чение 30). 
  - Возвращаемый тип данных - tuple с интервальной оценкой в формате (lower_bound, upper_bound).

### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

## Знакомство с данными